In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
os.environ["WANDB_DISABLED"] = "true"

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install transformers datasets peft trl

In [9]:
 from transformers import (
     AutoModelForCausalLM,
     AutoTokenizer,
     BitsAndBytesConfig,
     TrainingArguments,
   
 )
 import torch
 from datasets import load_dataset
 from peft import LoraConfig, get_peft_model
 from trl import SFTTrainer

In [10]:
# pip install -U transformers


In [ ]:
from huggingface_hub import login

# paste your HF token here
login("your hugging face token here")


In [12]:
 model_name = "meta-llama/Llama-3.2-1B-Instruct"

 # Tokenizer setup
 tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
 tokenizer.pad_token = tokenizer.eos_token
 tokenizer.padding_side = "right"


model = AutoModelForCausalLM.from_pretrained(model_name,
     device_map="auto"
 )


In [ ]:
# !pip install peft

In [13]:
 from peft import LoraConfig, get_peft_model

 lora_config = LoraConfig(
     lora_alpha=2048,
     lora_dropout=0.5,
     r=1024,
     bias="none",
     task_type="CAUSAL_LM",
 )

 modelT = get_peft_model(model, lora_config)

In [14]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("boltuix/emotions-dataset")

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Label'],
        num_rows: 131306
    })
})

In [16]:

# Assuming your dataset is called `ds`
columns_to_keep = [ "Sentence", "Label"]
# columns_to_remove = [c for c in ds_train.column_names if c not in columns_to_keep]

colums_to_remove=[c for c in ds['train'].column_names if c not in columns_to_keep]

ds_train = ds['train'].remove_columns(colums_to_remove)

print(ds_train)

Dataset({
    features: ['Sentence', 'Label'],
    num_rows: 131306
})


In [17]:
# new_ds=[]
# for i in ds_train:
#     print(i)
#     print['']
#     break

In [18]:
ds_train[0]

{'Sentence': 'Unfortunately later died from eating tainted meat NAME BBC documentary dynasties followed the marsh pride the lion episode was awesome',
 'Label': 'happiness'}

In [20]:
def preprocess_function(example):

    return {
        "prompt": [{"role": "user", "content":f"detect the emotion here:{example['Sentence']}" }],
        "completion": [
            {"role": "assistant", "content": example['Label']}
        ],
    }

dataset = ds_train.map(preprocess_function,remove_columns= ['Sentence','Label'])
print(dataset[0])






Map:   0%|          | 0/131306 [00:00<?, ? examples/s]

{'prompt': [{'content': 'detect the emotion here:Unfortunately later died from eating tainted meat NAME BBC documentary dynasties followed the marsh pride the lion episode was awesome', 'role': 'user'}], 'completion': [{'content': 'happiness', 'role': 'assistant'}]}


In [28]:
dataset[4]


{'prompt': [{'content': 'detect the emotion here:feel melty miserable enough imagine must',
   'role': 'user'}],
 'completion': [{'content': 'sadness', 'role': 'assistant'}]}

In [27]:
tokenizer.decode(tokenizer.encode(dataset[4]['completion'][0]['content']))

SyntaxError: incomplete input (305902809.py, line 1)

In [23]:
 from transformers import Trainer, TrainingArguments
 from trl import SFTTrainer

 training_args = TrainingArguments(
     output_dir="./results",
     eval_strategy="no",
     learning_rate=2e-5,
     per_device_train_batch_size=2,
     num_train_epochs=1,
     # dataset_text_field="text",
     fp16=True,
     save_total_limit=2,
     logging_steps=1000,
     save_steps=2500,

 )

 trainer = SFTTrainer(
     model= model,
     train_dataset=dataset,
    #  peft_config= lora_config,
     # dataset_text_field="text",
     # max_seq_length=512,  # Set max sequence length
     # tokenizer=tokenizer,
     args=training_args,
     # packing=False,
 )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Tokenizing train dataset:   0%|          | 0/131306 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/131306 [00:00<?, ? examples/s]

In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [25]:
trainer.train()

Step,Training Loss
1000,0.297300
2000,0.231300
3000,0.221300
4000,0.203700
5000,0.187600
6000,0.197800
7000,0.193600
8000,0.180800
9000,0.183900
10000,0.173100


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

KeyboardInterrupt: 

In [45]:
import time
for i in range(1000000):
    print(f"Heartbeat {i}")
    time.sleep(60)  # print every 1 minute


Heartbeat 0
Heartbeat 1
Heartbeat 2
Heartbeat 3
Heartbeat 4
Heartbeat 5
Heartbeat 6
Heartbeat 7
Heartbeat 8
Heartbeat 9
Heartbeat 10
Heartbeat 11
Heartbeat 12
Heartbeat 13
Heartbeat 14
Heartbeat 15
Heartbeat 16
Heartbeat 17
Heartbeat 18
Heartbeat 19
Heartbeat 20
Heartbeat 21
Heartbeat 22
Heartbeat 23
Heartbeat 24
Heartbeat 25
Heartbeat 26
Heartbeat 27


KeyboardInterrupt: 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [33]:
output_dir = "/kaggle/working/results/final"
model.save_pretrained(output_dir)

model.save_pretrained(output_dir)

In [47]:
model.push_to_hub("Subham-001/llama3.2_1B_emotion")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...py/model-00001-of-00002.safetensors:   1%|          | 41.9MB / 4.97GB            

  ...py/model-00002-of-00002.safetensors:   0%|          | 1.18MB /  405MB            

CommitInfo(commit_url='https://huggingface.co/Subham-001/llama3.2_1B_emotion/commit/241c7f5c2221124c137d059634031de5f0060a25', commit_message='Upload LlamaForCausalLM', commit_description='', oid='241c7f5c2221124c137d059634031de5f0060a25', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Subham-001/llama3.2_1B_emotion', endpoint='https://huggingface.co', repo_type='model', repo_id='Subham-001/llama3.2_1B_emotion'), pr_revision=None, pr_num=None)

In [52]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)
prompt = [{'content': 'please leave me alone ','role': 'user'}]
response = pipe(prompt)
response[0]["generated_text"]

Device set to use cuda:0


[{'content': 'please leave me alone ', 'role': 'user'},
 {'role': 'assistant', 'content': 'sadness'}]